In [1]:
import sys
root = '/home/featurize/data'
sys.path.append(f'{root}/pikachu/utils')
sys.path.append(f'{root}/pikachu/third')
sys.path.append(f'{root}/pikachu')
sys.path.append(f'{root}/pikachu/projects/ai/naic_seg')

In [2]:
import sys 
import os
import glob
import cv2
from PIL import Image
import numpy as np

from gezi import tqdm
from gezi.metrics.image.semantic_seg import Evaluator
import gezi

In [3]:
CLASSES_V2 = ['water', 'track_road', 'build', 'track_airport', 'other_park', 'other_playground', 
            'arable_natural', 'arable_greenhouse',
            'grass_natural', 'grass_greenbelt', 
            'forest_natural', 'forest_planted', 
            'bare_natural', 'bare_planted', 'other_other']
CLASSES = CLASSES_V2
NUM_CLASSES = len(CLASSES)
m = {}
for i in range(17):
  if i < 4:
    m[i + 1] = i
  else:
    m[i + 1] = i - 2

In [4]:
def eval(pred_dir, label_dir='../input/eval.naic/label'):
  key_metric = 'FWIoU'
  pred_files = glob.glob(f'{pred_dir}/*')

  evaluator = Evaluator(CLASSES)
  class_mious = 0.

  t = tqdm(enumerate(pred_files), total=len(pred_files), ascii=True, desc= 'eval')
  for i, pred_file in t:
    file_name = os.path.basename(pred_file)
    evaluator2 = Evaluator(CLASSES)
    label_file = f'{label_dir}/{file_name}'
    label = np.asarray(Image.open(label_file))
    pred = cv2.imread(pred_file,cv2.IMREAD_UNCHANGED).astype(np.uint8)
    
    f = np.vectorize(lambda x: m[x])
    label = f(label).astype(np.uint8)

    evaluator.eval_each(label[np.newaxis,:, :], pred[np.newaxis,:, :], metric=key_metric)
#     evaluator2.eval_each(label[np.newaxis,:, :], pred[np.newaxis,:, :], metric=key_metric)

    binary_label = np.bincount(label.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
    binary_pred = np.bincount(pred.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
    intersections = binary_label * binary_pred
    intersection = np.sum(intersections, axis=-1)
    union = np.sum(binary_label, axis=-1) + np.sum(binary_pred, axis=-1) - intersection
    class_miou = np.mean(intersection / union)

    class_mious += class_miou

    metric = evaluator.eval_once(key_metric)
#     metric2 = evaluator2.eval_once(key_metric)
    t.set_postfix({key_metric: metric, 'IMAGE/CLASS/MIoU': class_mious / (i + 1)})
    
#     if metric2 < 0.1:
#       print(file_name, metric2, class_miou)

  res = evaluator.eval_once()
  res['IMAGE/CLASS/MIoU'] = class_mious / i
  gezi.set('evaluator', evaluator)
  
  return res

In [5]:
eval('../input/out')

{'FWIoU': 0.7778595563053591,
 'MIoU': 0.7115360885254554,
 'ACC/pixel': 0.8727666595458984,
 'ACC/class': 0.7970300189375601,
 'IoU/water': 0.7916577813042049,
 'IoU/track_road': 0.8048845182297052,
 'IoU/build': 0.865484699458146,
 'IoU/track_airport': 0.0,
 'IoU/other_park': 0.8718514392838806,
 'IoU/other_playground': 0.8283863961009432,
 'IoU/arable_natural': 0.8506058340246935,
 'IoU/arable_greenhouse': 0.7910985606788657,
 'IoU/grass_natural': 0.652738091659201,
 'IoU/grass_greenbelt': 0.7659750800417351,
 'IoU/forest_natural': 0.7466731268339089,
 'IoU/forest_planted': 0.637545932904967,
 'IoU/bare_natural': 0.8054980806120025,
 'IoU/bare_planted': 0.7166763397989279,
 'IoU/other_other': 0.5439654469506475,
 'IMAGE/CLASS/MIoU': 0.886716824640617}

In [6]:
eval('../input/out.2')

{'FWIoU': 0.7780047957622916,
 'MIoU': 0.7109949620525206,
 'ACC/pixel': 0.8728516265869141,
 'ACC/class': 0.7963652543237473,
 'IoU/water': 0.7921528853062886,
 'IoU/track_road': 0.8050809563510685,
 'IoU/build': 0.865668890762374,
 'IoU/track_airport': 0.0,
 'IoU/other_park': 0.8724400013862645,
 'IoU/other_playground': 0.8200995757981693,
 'IoU/arable_natural': 0.8506646139129812,
 'IoU/arable_greenhouse': 0.7912588326002139,
 'IoU/grass_natural': 0.6530510552586739,
 'IoU/grass_greenbelt': 0.766720763250973,
 'IoU/forest_natural': 0.7470513143552305,
 'IoU/forest_planted': 0.6378407488937464,
 'IoU/bare_natural': 0.8047805877303398,
 'IoU/bare_planted': 0.7147780798927748,
 'IoU/other_other': 0.5433361252887112,
 'IMAGE/CLASS/MIoU': 0.8867348423153997}

In [7]:
eval('../input/out.3')

{'FWIoU': 0.7790593023149208,
 'MIoU': 0.7123095990480225,
 'ACC/pixel': 0.8734884094238281,
 'ACC/class': 0.7972925255933666,
 'IoU/water': 0.7948946767854415,
 'IoU/track_road': 0.8077832908836104,
 'IoU/build': 0.8669825589089382,
 'IoU/track_airport': 0.0,
 'IoU/other_park': 0.874072627310912,
 'IoU/other_playground': 0.8276761422607447,
 'IoU/arable_natural': 0.852054759044725,
 'IoU/arable_greenhouse': 0.7938192194143369,
 'IoU/grass_natural': 0.6543796182831398,
 'IoU/grass_greenbelt': 0.7614490638649299,
 'IoU/forest_natural': 0.7466760521367689,
 'IoU/forest_planted': 0.6385844402083368,
 'IoU/bare_natural': 0.8060535109834166,
 'IoU/bare_planted': 0.7161200475559886,
 'IoU/other_other': 0.5440979780790498,
 'IMAGE/CLASS/MIoU': 0.888313344321445}

In [9]:
eval('../input/out.4')

{'FWIoU': 0.7803804159874678,
 'MIoU': 0.7131189104517832,
 'ACC/pixel': 0.8743195175170898,
 'ACC/class': 0.7973623439958104,
 'IoU/water': 0.7949127501202037,
 'IoU/track_road': 0.8093785373432069,
 'IoU/build': 0.8678973762402427,
 'IoU/track_airport': 0.0,
 'IoU/other_park': 0.8760099070768841,
 'IoU/other_playground': 0.8167003393438579,
 'IoU/arable_natural': 0.852871926368416,
 'IoU/arable_greenhouse': 0.7959723189361401,
 'IoU/grass_natural': 0.6554955856704598,
 'IoU/grass_greenbelt': 0.7705801719159979,
 'IoU/forest_natural': 0.7476955965369697,
 'IoU/forest_planted': 0.6413286546430517,
 'IoU/bare_natural': 0.806909103894082,
 'IoU/bare_planted': 0.7165366198764253,
 'IoU/other_other': 0.5444947688108116,
 'IMAGE/CLASS/MIoU': 0.8886501312468894}